In [6]:
# This notebook requires an unreleased version of sparse which exists in GitHub
import numpy as np
import sparse

import time

In [1]:
def compare_bounds(l1,l2):
    N1 = l1.shape[0]
    N2 = l2.shape[0]
    i = 0
    j = 0

    sparse_data = []
    sparse_inds = []

    while i < N1 and j < N2:
        inds = (i,j)
        ta = l2[j,1] - l2[j,0]

        if l1[i,1] < l2[j,1]:
            if l1[i,1] < l2[j,0]:
                so = 0
            elif l1[i,0] < l2[j,0]:
                so = l1[i,1] - l2[j,0]
            else:
                so = l1[i,1] - l1[i,0]
            i += 1
        elif l1[i,1] > l2[j,1]:
            if l1[i,0] > l2[j,1]:
                so = 0
            elif l1[i,0] < l2[j,0]:
                so = l2[j,1] - l2[j,0]
            else:
                so = l2[j,1] - l1[i,0]
            j += 1
        else:
            if l1[i, 0] < l2[j, 0]:
                so = l2[j, 1] - l2[j, 0]
            else:
                so = l2[j, 1] - l1[i, 0]
            i += 1
            j += 1

        frac = so/ta
        sparse_data.append(frac)
        sparse_inds.append(inds)

    coords = np.array(sparse_inds).T
    data = np.array(sparse_data)
    shape = (N1, N2)
    sp_array = sparse.COO(coords, data=data, shape=shape)
    return sp_array

In [2]:
def combine_arrays(lon_array, lat_array):
    sx, tx = lon_array.shape
    sy, ty = lat_array.shape
    combo_array = sparse.outer(lon_array, lat_array).transpose([2,0,3,1]).reshape([sx*sy, tx*ty])
    return combo_array.tocsr()

In [3]:
def find_weights(lons1, lons2, lats1, lats2):
    
    # Account for spherical geometry
    transformed_lat1 = np.sin(lats1*np.pi/180)
    transformed_lat2 = np.sin(lats2*np.pi/180)
    
    # If grids are cartesian
    #
    # transformed_lat1 = lats1
    # transformed_lat2 = lats2
    
    lat_array = compare_bounds(transformed_lat1,transformed_lat2)
    lon_array = compare_bounds(lons1, lons2)
    
    combo = combine_arrays(lon_array, lat_array)
    return combo

In [4]:
def regrid(array, lons1, lons2, lats1, lats2):
    x = lons2.shape[0]
    y = lats2.shape[0]

    weights = find_weights(lons1, lons2, lats1, lats2)

    flat_out = weights.T*array.flatten()[:,np.newaxis]
    out = flat_out.reshape([x, y])
    return out

In [7]:
# Construct lat-lon bounds

x1 = 1000
y1 = 1200

x2 = 2000
y2 = 3000

lons1 = np.linspace(-180,180,x1 +1)
lonbnds1 = np.stack([lons1[:-1],lons1[1:]]).T
lons2 = np.linspace(-180,180,x2 +1)
lonbnds2 = np.stack([lons2[:-1],lons2[1:]]).T


lats1 = np.linspace(-90,90,y1 +1)
latbnds1 = np.stack([lats1[:-1],lats1[1:]]).T
lats2 = np.linspace(-90,90,y2 +1)
latbnds2 = np.stack([lats2[:-1],lats2[1:]]).T

# Construct data to regrid

array = np.array(range(x1*y1)).reshape([x1,y1])

In [ ]:
# Perform regridding

out = regrid(array,lonbnds1,lonbnds2,latbnds1,latbnds2)